**Check for cuda**

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# import torch
# from torchvision.models import vgg16, VGG16_Weights

# # Ensure that device is defined (e.g., 'cpu' or 'cuda')
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Load the VGG16 model with pretrained weights
# model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1).to(device)

**Down Data**

In [4]:
# import requests
# import zipfile
# from pathlib import Path

# data_path = Path("data/")
# image_path = data_path / "pizza_steak_sushi"

# if image_path.is_dir():
#     print(f"{image_path} directory exists.")
# else:
#     print(f"Did not find {image_path} directory, creating one...")
#     image_path.mkdir(parents=True, exist_ok=True)

#     with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
#         request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
#         print("Downloading pizza, steak, sushi data...")
#         f.write(request.content)

#     with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
#         print("Unzipping pizza, steak, sushi data...")
#         zip_ref.extractall(image_path)

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
# unzip = 0
# if unzip == 0:
#   !unzip /content/drive/MyDrive/Animal_dataset/VGG_16.zip -d /content/animal_dataset
#   unzip += 1
# else:
#   print("already zip")

**Build Custome Dataset**

In [7]:
import os
import pathlib
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, datasets
from torchvision.transforms import Resize, ToTensor, Compose, Normalize, RandomHorizontalFlip
from torchvision.models import resnet34, ResNet34_Weights
from PIL import Image
from typing import Tuple, Dict, List
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import argparse
import cv2
import shutil



In [8]:
def find_classes(directory: str) -> Tuple[List[str], Dict[str, int]]:

    classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())
    if not classes:
        raise FileNotFoundError(f"Couldn't find any classes in {directory}.")

    class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
    return classes, class_to_idx

In [9]:

class ImageFolderCustom(Dataset):
  def __init__(self, root_dir, is_train, transform):
    if is_train:
      data_path = os.path.join(root_dir, "train")
    else:
      data_path = os.path.join(root_dir, "test")
    self.root_dir = root_dir
    self.categories = ['cow', 'horse', 'sheep']
    self.image_paths = []
    self.labels = []
    self.transform = transform
    for index, category in enumerate(self.categories):
      subdir_path = os.path.join(data_path, category)
      for file_name in os.listdir(subdir_path):
          self.image_paths.append(os.path.join(subdir_path,subdir_path, file_name))
          self.labels.append(index)

  def __len__(self) -> int:
      "Returns the total number of samples."
      return len(self.image_paths)

  def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
      "Returns one sample of data, data and label (X, y)."
      image = Image.open(self.image_paths[index]).convert("RGB")
      class_idx = self.labels[index]
      if self.transform:
          image = self.transform(image)
      return image, class_idx

In [10]:
root_dir = r'PATH TO YOUR DATASET FOLDER'

In [11]:
def data_loader(root_dir:str) -> tuple[DataLoader,DataLoader]:
  train_transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    RandomHorizontalFlip(p=0.5),
    Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010])
    ])
  test_transform = Compose([
    Resize((224, 224)),
    ToTensor()
    ])

  train_dataset = ImageFolderCustom(root_dir=root_dir, is_train=True, transform=train_transform)
  train_dataloader = DataLoader(
      dataset = train_dataset,
      batch_size = 4,
      num_workers = 4,
      shuffle = True,
      drop_last = False
  )
  test_dataset = ImageFolderCustom(root_dir=root_dir, is_train=False, transform=test_transform)
  test_dataloader = DataLoader(
        dataset = test_dataset,
        batch_size = 2,
        num_workers = 2,
        shuffle = False,
        drop_last = False
  )
  return train_dataloader, test_dataloader

train_dataloader, test_dataloader = data_loader(root_dir)

In [12]:
# TEST IMAGE INFO AFTER DATALOADER

# # train_transform = Compose([
# #     Resize((224, 224)),
# #     ToTensor(),
# #     RandomHorizontalFlip(p=0.5),
# #     Normalize(
# #         mean=[0.4914, 0.4822, 0.4465],
# #         std=[0.2023, 0.1994, 0.2010])
# #     ])
# # train_dataset = ImageFolderCustom(root_dir=root_dir, is_train=True, transform=train_transform)
# # images_neduma,labels_neduma = train_dataset.__getitem__(200)
# # print(images_neduma.shape)
# # print(labels_neduma)

In [13]:
#CHECK CLASSES

# from PIL import Image
# import matplotlib.pyplot as plt
# categories = ['pizza', 'steak', 'sushi']
# data = ImageFolderCustom(root_dir=root_dir,  is_train=True)

# image,label = data.__getitem__(200)

# img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# im_pil = Image.fromarray(img)

# im_np = np.asarray(im_pil)

# plt.imshow(im_np)
# plt.title(categories[label])
# plt.axis('off')
# plt.show()


**MODEL**

In [14]:
IMAGE_SIZE = 224
NUM_CLASSES = 3

NUM_EPOCHS = 100
NUM_WORKERS = 4
LEARNING_RATE = 0.001

CONV_KERNEL = 3
CONV_STRIDE = 1
CONV_PADDING = 1
MP_KERNEL = 2
MP_STRIDE = 2
MP_PADDING = 0

VGG16_ARCHITECTURE = [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"]

In [15]:
class VGG(nn.Module):
  def __init__(self, in_channels=3, num_classes=None):
    super(VGG, self).__init__()
    self.in_channels = in_channels
    self.features = self.create_conv_layers(VGG16_ARCHITECTURE)
    self.avgpool = nn.AdaptiveAvgPool2d((7,7))
    self.classifier = nn.Sequential(
        nn.Linear(512*7*7, 4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 1000),
        nn.Linear(1000,512),
        nn.Linear(512, num_classes),
    )

  def forward(self,x):
    x = self.features(x)
    x = x.reshape(x.shape[0], -1)
    x = self.classifier(x)
    return x

  def create_conv_layers(self, archite):
    layers = []
    in_channels = self.in_channels
    for x in archite:
      if type(x) == int:
        out_channels = x
        layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=(CONV_KERNEL,CONV_KERNEL), stride=(CONV_STRIDE,CONV_STRIDE), padding=(CONV_PADDING,CONV_PADDING)),
                  #  nn.BatchNorm2d(x),
                   nn.ReLU()]
        in_channels = x
      elif x == "M":
        layers += [nn.MaxPool2d(kernel_size=(MP_KERNEL,MP_KERNEL), stride=(MP_STRIDE,MP_STRIDE))]
    return nn.Sequential(*layers)


In [16]:
print("My VGG16 - achritecture")
model = VGG(in_channels=3, num_classes=NUM_CLASSES).to(device=device)
state_dict = torch.load(r'PATH TO YOUR VGG16_PRETRAINED WEIGHT')
model.load_state_dict(state_dict, strict= False)
print(model)

My VGG16 - achritecture
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU()
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (17): C

**LOST - OPTIMIZER**

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

**Train/Test Phrase**

In [18]:
stored_dir = r'PATH TO YOUR FOLDER TO STORE THE TENSORBOARD LOG'
check_point = r'YOUR PATH TO YOUR MODE_WEIGHT.pt'

In [20]:
def main():
  train()

In [21]:
main()

100%|██████████| 315/315 [00:10<00:00, 29.19it/s]


Epoch 1/100. Loss 0.2638. Acc 0.8873


100%|██████████| 315/315 [00:10<00:00, 28.88it/s]


Epoch 2/100. Loss 0.1812. Acc 0.9286


100%|██████████| 315/315 [00:10<00:00, 29.00it/s]


...

Epoch 19/100. Loss 0.1348. Acc 0.9540


100%|██████████| 315/315 [00:10<00:00, 30.23it/s]


Epoch 20/100. Loss 0.1868. Acc 0.9397


Epoch 21/100. Loss 0.0000:  73%|███████▎  | 1043/1423 [02:46<01:00,  6.25it/s]


KeyboardInterrupt: 